In [3]:
!pip install Faker
!pip install pandas 
!pip install tweepy
!pip install confluent_kafka

     |████████████████████████████████| 2.7 MB 1.3 MB/s eta 0:00:01


In [4]:
import time

from confluent_kafka import Producer
import json
import tweepy

producer = Producer(**{"bootstrap.servers": "kafka2:19092"})

In [5]:
consumer_key = "hK08N3zIwkJLrF6BVK52b3aS8"
consumer_secret = "8M22ppkZzb3sC9u4acUZK5IBr9nsLSByctTOoUI3Y5THEMRk7s"
access_token = "774855303321886720-JxdOM3ieqEVck6l0rYJmSIV3A412anW"
access_token_secret = "AoJgcvbtWmikzHaoOwH2rURrzoRwbBGgoV4oEASumNMzF"

In [6]:
# Words to track
WORDS = ['#coronavirus', '#COVID-19', '#COVID19', '#COVID'] #, '#SocialDistancing', '#pandemic']
raw_tweets_topic = "1_RAW_tweets"

In [7]:
class StreamListener(tweepy.StreamListener):
    # This is a class provided by tweepy to access the Twitter Streaming API.

    def on_connect(self):
        # Called initially to connect to the Streaming API
        print("You are now connected to the streaming API.")

    def on_error(self, status_code):
        # On error - if an error occurs, display the error / status code
        print("Error received in kafka producer " + repr(status_code))
        
        return True 
    
    def preprocess(self, tweetData):
        userID = '-'
        tweetText = '-'
        hashTags = []
        location_full_name = '-'
        location_coutry = '-'
        created_at = '-'

        try:
            userID = tweetData["user"]["id"]
            
            if tweetData["truncated"] == True:
                tweetText = tweetData["extended_tweet"]["full_text"]
                hashTags = [ hTag["text"] for hTag in tweetData["extended_tweet"]["entities"]["hashtags"] ]
            else:
                tweetText = tweetData["text"]
                hashTags = [ hTag["text"] for hTag in tweetData["entities"]["hashtags"] ]

            if hashTags == []:
                return {}

            if "place" in tweetData and tweetData["place"] != None:
                location_full_name = tweetData["place"]["full_name"]
                location_coutry = tweetData["place"]["country"]
                
            created_at = tweetData["created_at"]

            rec = {'userID': userID, 
                   'tweetText': tweetText, 
                   'hashTags': hashTags, 
                   'location_full_name': location_full_name,
                   'location_coutry': location_coutry,
                   'created_at': created_at }

            return json.dumps(rec)

        except Exception as ex:
            print('Exception while parsing')
            print(str(ex))

    def on_data(self, data):
        """ This method is called whenever new data arrives from live stream.
        We asynchronously push this data to kafka queue"""
        try:
            parsed = json.loads(data)

            
            if "user" in parsed and "location" in parsed["user"]:
                if parsed["user"]["location"] != None:
                    processed = self.preprocess(parsed)
                    
                    if isinstance(processed, str):
                        producer.produce(raw_tweets_topic, value=processed.encode('utf-8'))
                        producer.flush()

                        print(parsed["text"])

        except Exception as e:
            print("error! : " + str(e))
            return False #stop stream

        return True

    def on_timeout(self):
        return True

In [8]:
# Create Auth object
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

# Set up the listener. The 'wait_on_rate_limit=True' is needed to help with Twitter API rate limiting.
listener = StreamListener(api=tweepy.API(wait_on_rate_limit=True, wait_on_rate_limit_notify=True, timeout=60, retry_delay=5, retry_count=10, retry_errors=set([401, 404, 500, 503])))
stream = tweepy.Stream(auth=auth, listener=listener)
print("Tracking: " + str(WORDS))
stream.filter(track=WORDS, languages = ['en'])

Tracking: ['#coronavirus', '#COVID-19', '#COVID19', '#COVID']
You are now connected to the streaming API.
RT @hiral4congress: It’s breathtaking how fiercely @GOP governors are fighting #COVID vaccine and mask mandates w/arguments of “personal li…
#BREAKING: Seven countries have been added to the #UK's Green travel list, meaning holidaymakers will be able to re… https://t.co/YU8P8oULbX
RT @ICMRDELHI: .@harbhajan_singh joins the fight against #COVID19! Harbhajan Singh, noted cricketer, met ICMR team today to offer his suppo…
RT @NathanStall: All 34 Medical Officers of Health agree that #COVID19 vaccine passports are needed.

There is also overwhelming public sup…
Is the use of virtual applications, particularly social media, for self-growth counter-productive for companies tha… https://t.co/0ODbuSYNOa
As @abc15 states, #Arizona is facing high #COVID19 cases!

Learn about your county's vulnerability with @CARESMU CO… https://t.co/A7R0jMH0r3
Dome is Ready!
Now, up to 50% discounts! https:/

At least 50% of the total adult population of India has received their first dose of the Covid-19 vaccine, said Uni… https://t.co/59IuJrmfXT
RT @jennycohn1: The GOP is weaponizing the #COVID19 pandemic to take over &amp; destroy public education, one of their long term goals. It is h…
RT @Jamz5251: Daily death toll passes 200 mark for the first time... 

209 #COVID19 deaths reported in #SriLanka yesterday increasing the t…
RT @CBCLorenda: Here's a look at hospitalizations in Ontario due to #Covid19:
302 in hospital (+19)
165 in ICU (+4)
89 on a ventilator (-3)
RT @iran_policy: Iran Covid-19 Crisis: State Media Acknowledge the Death Toll is above 700,000
#Iran #COVID19 #CoronavirusUpdates  https://…
RT @NC5PhilWilliams: This morning, 60 Tennessean children are hospitalized with #COVID19. 16 are in ICU, 6 are on ventilators. Last week, 1…
.@AirCanada is making it mandatory for all employees to be fully vaccinated against #COVID19 &amp; report their vaccina… https://t.co/flB412yaDZ
RT @Ma

KeyboardInterrupt: 